In [1]:
import pandas as pd

df = pd.read_csv("/Users/robinsuxdorf/Downloads/archive-4/job_skills.csv")
df2 = pd.read_csv("/Users/robinsuxdorf/Downloads/archive-4/linkedin_job_postings.csv")

In [ ]:
merge_df = pd.merge(df, df2, on="job_link", how="inner")

In [ ]:
job_df = merge_df[["job_link", "job_skills", "job_title", "company", "job_location", "job_level"]]

In [ ]:
job_df = job_df[job_df["job_skills"].isna() == False]
job_df = job_df[job_df["job_title"].isna() == False]
job_df = job_df[job_df["company"].isna() == False]

job_df = job_df.head(100)
job_df.reset_index(drop=True, inplace=True)

In [ ]:
skills: list[str] = []
job_titles: list[str] = []
companies: list[str] = []

for index, row in job_df.iterrows():
    job_skills = row["job_skills"].split(",")
    skills.extend(job_skills)

    job_titles.append(row["job_title"])
    companies.append(row["company"])

skills = list(set(skills))
job_titles = list(set(job_titles))
companies = list(set(companies))

In [ ]:
## check for lowercase
## remove unnecessary whitespaces
## remove skills with numbers?

# def remove_duplicates(words):
#     seen = set()
#     unique_words = []
    
#     for word in words:
#         normalized_word = word.lower()
#         if normalized_word not in seen:
#             seen.add(normalized_word)
#             unique_words.append(word)
    
#     return unique_words

# skills = remove_duplicates(skills)

# len(skills)

In [ ]:
skill_to_idx = {skill: index for index, skill in enumerate(skills)}
job_title_to_idx = {job_title: index for index, job_title in enumerate(job_titles)}
company_to_idx = {company: index for index, company in enumerate(companies)}

In [ ]:
def map_skills_to_ids(row):

    job_skills = row.split(",")

    skill_ids = [skill_to_idx[skill] for skill in job_skills]
    return skill_ids

In [ ]:
job_df["skill_ids"] = job_df["job_skills"].apply(map_skills_to_ids)
job_df["job_title_id"] = job_df["job_title"].apply(lambda job: job_title_to_idx[job])
job_df["company_id"] = job_df["company"].apply(lambda company: company_to_idx[company])

In [ ]:
job_df

In [6]:
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "RS96ltme44ever")

In [ ]:
from neo4j import GraphDatabase

## add contraints for other entities
## add skill relationships
## extract queries

def setup_index(driver: GraphDatabase.driver) -> None:
    with driver.session() as session:
        session.run("CREATE CONSTRAINT FOR (s:Skill) REQUIRE s.skillID IS UNIQUE")

def add_skills(driver: GraphDatabase.driver, skill_to_idx: tuple[str, int]) -> None:
    with driver.session() as session:
        with session.begin_transaction() as tx:
            for skill, skill_id in skill_to_idx.items():
                tx.run(
                    "MERGE (:Skill {skillID: $skillID, skillName: $skillName})",
                    skillID=skill_id, skillName=skill
                )
            tx.commit()

def add_companies(driver: GraphDatabase.driver, company_to_idx: tuple[str, int]) -> None:
    with driver.session() as session:
        with session.begin_transaction() as tx:
            for company, company_id in company_to_idx.items():
                tx.run(
                    "MERGE (:Company {companyID: $companyID, companyName: $companyName})",
                    companyID=company_id, companyName=company
                )
            tx.commit()

def add_job_titles(driver: GraphDatabase.driver, job_title_to_idx: tuple[str, int]) -> None:
    with driver.session() as session:
        with session.begin_transaction() as tx:
            for job_title, job_title_id in job_title_to_idx.items():
                tx.run(
                    "MERGE (:JobTitle {jobTitleID: $jobTitleID, jobTitle: $jobTitle})",
                    jobTitleID=job_title_id, jobTitle=job_title
                )
            tx.commit()

def add_jobs(driver: GraphDatabase, df: pd.DataFrame) -> None:
    with driver.session() as session:
        with session.begin_transaction() as tx:
            for _, job in df.iterrows():
                tx.run(
                    """
                        MATCH (c:Company {companyID: $companyID})
                        MATCH (jt: JobTitle {jobTitleID: $jobTitleID})

                        MERGE (j:Job {link: $link})
                        MERGE (j)-[:HAS_ROLE]->(jt)
                        MERGE (j)-[:FOR_COMPANY]->(c)
                    """,
                    companyID=job["company_id"],
                    jobTitleID=job["job_title_id"],
                    link=job["job_link"]
                )

URI = "neo4j://localhost:7687"
URI2 = "bolt://localhost:7687"
AUTH = ("neo4j", "RS96ltme44ever")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    # setup_index(driver)
    # add_skills(driver, skill_to_idx)
    add_companies(driver, company_to_idx)
    add_job_titles(driver, job_title_to_idx)
    add_jobs(driver, job_df)

In [8]:
from graphdatascience import GraphDataScience

gds = GraphDataScience(endpoint=URI, auth=AUTH)

In [26]:
gds.graph.drop("skill_graph")

graphName                                                      skill_graph
database                                                             neo4j
databaseLocation                                                     local
memoryUsage                                                               
sizeInBytes                                                             -1
nodeCount                                                             1658
relationshipCount                                                        0
configuration            {'relationshipProjection': {'HAS_ROLE': {'aggr...
density                                                                0.0
creationTime                           2024-05-28T20:33:51.489454000+00:00
modificationTime                       2024-05-28T20:33:51.553327000+00:00
schema                   {'graphProperties': {}, 'nodes': {'Skill': {}}...
schemaWithOrientation    {'graphProperties': {}, 'nodes': {'Skill': {}}...
Name: 0, dtype: object

In [27]:
G, result = gds.graph.project(
    "job_graph",
    ["Job", "JobTitle", "Company"],
    ["HAS_ROLE", "FOR_COMPANY"]
)

In [28]:
G.node_count()

285